# Final Project - Reinforcements Learning 
Hello dear students,<br> this is the template notebook. Please click on the "File" tab and then on "Save a copy into drive".

---
<br>

### Name and ID:
Student 1: Avraham Raviv, 204355390
<br>
Student 2: Yevgeni Berkovitch, 317079234
<br><br>
<img src="https://play-lh.googleusercontent.com/e_oKlKPISbgdzut1H9opevS7-LTB8-8lsmpCdMkhlnqFenZhpjxbLmx7l158-xQQCIY">

### https://github.com/mpSchrader/gym-sokoban

# Installs

In [1]:
%%capture
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install gym
!pip install pygame
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install gym_sokoban

!imageio_download_bin ffmpeg

# Imports

In [2]:
import random
import time

import numpy as np
import matplotlib.pyplot as plt

import base64
import imageio
from pyvirtualdisplay import Display
from IPython.display import HTML

import gym
from gym import error, spaces, utils
from soko_pap import *

from collections import deque
from queue import PriorityQueue

from keras.models import Sequential
from keras.layers import Dense

In [3]:
%matplotlib inline

In [4]:
imageio.plugins.ffmpeg.download()

In [5]:
from gym import logger as gymlogger
gymlogger.set_level(40) # error only

# Display utils
The cell below contains the video display configuration. No need to make changes here.

In [6]:
def embed_mp4(filename):
    """Embeds an mp4 file in the notebook."""
    video = open(filename,'rb').read()
    b64 = base64.b64encode(video)
    tag = '''
    <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>'''.format(b64.decode())

    return HTML(tag)

# Utils

In [7]:
def get_distances(room_state):
    for i in range(room_state.shape[0]):
        for j in range(room_state.shape[1]):
            if room_state[i][j] == 2:
                target = (i, j)

    distances = np.zeros(shape=room_state.shape)
    visited_cells = set()
    cell_queue = deque()

    visited_cells.add(target)
    cell_queue.appendleft(target)

    while len(cell_queue) != 0:
        cell = cell_queue.pop()
        distance = distances[cell[0]][cell[1]]
        for x,y in ((1,0), (-1,-0), (0,1), (0,-1)):
            next_cell_x, next_cell_y = cell[0]+x, cell[1]+y
            if room_state[next_cell_x][next_cell_y] != 0 and not (next_cell_x, next_cell_y) in visited_cells:
                distances[next_cell_x][next_cell_y] = distance + 1
                visited_cells.add((next_cell_x, next_cell_y))
                cell_queue.appendleft((next_cell_x, next_cell_y))
                
    return distances   

def calc_distances(room_state, distances):
    box = None
    mover = None
    for i in range(room_state.shape[0]):
        for j in range(room_state.shape[1]):            
            if room_state[i][j] == 4:
                box = (i,j)
            
            if room_state[i][j] == 5:
                mover = (i,j)
    
    return mover, box, distances[box[0]][box[1]]   

def box2target_change_reward(room_state, next_room_state, distances):
    mover, box, t2b = calc_distances(room_state, distances)
    n_mover, n_box, n_t2b = calc_distances(next_room_state, distances)
    
    change_reward = 0.0
    if n_t2b < t2b:
        change_reward += 1.0
    elif n_t2b > t2b:
        change_reward -= 1.0
        
    m2b = np.sqrt((mover[0]-box[0])**2 + (mover[1]-box[1])**2)
    n_m2b = np.sqrt((n_mover[0]-n_box[0])**2 + (n_mover[1]-n_box[1])**2)
    
    if n_m2b < m2b and m2b >= 2:
        change_reward += 0.25
    elif n_m2b > m2b and n_m2b >= 2:
        change_reward -= 0.25
        
    return change_reward   

# Solution

In [8]:
class SOK_Agent:
    def __init__(self):
        # Construct DQN models
        self.state_size = (25,) 
        self.action_size = 8
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.target_model.set_weights(self.model.get_weights())
        self.batch_size = 8
        
        # Replay buffers
        self.replay_buffer = deque(maxlen=1500)
        self.prioritized_replay_buffer = deque(maxlen=500)
        self.prioritized_replay_batch = 50        
        
        # Hyperparameters
        self.gamma = 0.9
        self.epsilon = 1.0   
        self.epsilon_min = 0.3
        self.epsilon_decay = 0.995
        self.epsilon_update_rate = 10
        self.replay_rate = 10
        self.update_beta = 0.99

        self.verbosity = 100 

    def _build_model(self):
        model = Sequential()
        model.add(Dense(512, input_shape=self.state_size, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer="adam")        
        return model

    def remember(self, state, action, reward, next_state, done):
        self.replay_buffer.append([state, action, reward, next_state, done])    
        
    def copy_to_prioritized_buffer(self, n):
        for i in range(n):
            self.prioritized_replay_buffer.append(self.replay_buffer[-1-i])  

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0]) 

    def replay(self):        
        minibatch = random.sample(self.replay_buffer, self.batch_size//2) 
        minibatch.extend(random.sample(self.prioritized_replay_buffer, self.batch_size//2))
        
        states = np.zeros((self.batch_size, self.state_size[0]))
        actions = np.zeros(self.batch_size, dtype=int)
        rewards = np.zeros(self.batch_size)
        next_states = np.zeros((self.batch_size, self.state_size[0]))
        statuses = np.zeros(self.batch_size)
        targets = np.zeros((self.batch_size, self.action_size)) 
        
        for i, (state, action, reward, next_state, done) in enumerate(minibatch): 
            states[i] = state.copy()
            actions[i] = action
            rewards[i] = reward
            next_states[i] = next_state.copy()
            statuses[i] = 1 if done else 0    
        
        targets = self.model.predict(states) 
        max_actions = np.argmax(self.model.predict(next_states), axis=1)
        next_rewards = self.target_model.predict(next_states)
        
        ind = 0
        for action, reward, next_reward, max_action, done in zip(actions, rewards, next_rewards, max_actions, statuses):  
            if not done:
                reward += self.gamma * next_reward[max_action]
            targets[ind][action] = reward
            ind += 1
        
        self.model.fit(states, targets, epochs=10, verbose=0) 
        
    def update_epsilon(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon = self.epsilon * self.epsilon_decay
        
    def update_target_model(self):
        model_w = self.model.get_weights()
        target_model_w = self.target_model.get_weights()
        updated_target_model_w = []
        for i in range(len(model_w)):
            updated_target_model_w.append(self.update_beta*target_model_w[i] + (1-self.update_beta)*model_w[i])
        self.target_model.set_weights(updated_target_model_w)    
            
    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [9]:
def process_frame(frame):
    f = frame[16:96, 16:96, 0]   
    f = f.reshape(5, 16, 5, 16).max(axis=(1, 3))
    f = f.flatten()
    f = f / 255
    return np.expand_dims(f, axis=0)

## Training

In [10]:
max_episodes = 10000
max_steps = 500

def init_sok(r):
    random.seed(r%10)
    sok = PushAndPullSokobanEnv(dim_room=(7, 7), num_boxes=1)
    sok.set_maxsteps(max_steps)
    return sok

In [11]:
agent = SOK_Agent()
successes_before_train = 20
successful_episodes = 0
continuous_successes_goal = 100
continuous_successes = 0

steps_per_episode = []

for e in range(max_episodes):
    if continuous_successes >= continuous_successes_goal:
        print("Agent training finished!")
        break
    
    print("Episode: %d" % (e))
    
    sok = init_sok(e)
    state = process_frame(sok.get_image('rgb_array'))
    random.seed(e)
    
    room_state = sok.room_state.copy() 
    distances = get_distances(room_state)
    
    for step in range(sok.max_steps):
        action = agent.act(state)
        if action < 4:
            next_state, reward, done, _ = sok.step(action+1) 
        else:
            next_state, reward, done, _ = sok.step(action+5)         
        
        next_state = process_frame(next_state)        
        next_room_state = sok.room_state
        
        if not done:
            reward += box2target_change_reward(room_state, next_room_state, distances)
        
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state.copy() 
        room_state = next_room_state.copy()
                
        if successful_episodes >= successes_before_train:
            if (step+1) % agent.replay_rate == 0:
                agent.replay() 
                agent.update_target_model()
                agent.update_epsilon()
        
        if done:            
            if 3 in sok.room_state:
                successful_episodes += 1
                continuous_successes += 1
                print("SOLVED! Episode %d Steps: %d Epsilon %.4f" % (e, step+1, agent.epsilon)) 
                agent.copy_to_prioritized_buffer(min(agent.prioritized_replay_batch, step+1))
            else:
                continuous_successes = 0
                
            steps_per_episode.append(step+1)
            #agent.save("exp1_episode%d.h5" % (e))
            
            break

Episode: 0
Episode: 1
SOLVED! Episode 1 Steps: 172 Epsilon 1.0000
Episode: 2
Episode: 3
SOLVED! Episode 3 Steps: 54 Epsilon 1.0000
Episode: 4
SOLVED! Episode 4 Steps: 1 Epsilon 1.0000
Episode: 5
SOLVED! Episode 5 Steps: 28 Epsilon 1.0000
Episode: 6
Episode: 7
SOLVED! Episode 7 Steps: 25 Epsilon 1.0000
Episode: 8
SOLVED! Episode 8 Steps: 352 Epsilon 1.0000
Episode: 9
SOLVED! Episode 9 Steps: 74 Epsilon 1.0000
Episode: 10
Episode: 11
SOLVED! Episode 11 Steps: 53 Epsilon 1.0000
Episode: 12
Episode: 13
Episode: 14
SOLVED! Episode 14 Steps: 2 Epsilon 1.0000
Episode: 15
Episode: 16
SOLVED! Episode 16 Steps: 138 Epsilon 1.0000
Episode: 17
SOLVED! Episode 17 Steps: 449 Epsilon 1.0000
Episode: 18
SOLVED! Episode 18 Steps: 6 Epsilon 1.0000
Episode: 19
Episode: 20
Episode: 21
SOLVED! Episode 21 Steps: 134 Epsilon 1.0000
Episode: 22
SOLVED! Episode 22 Steps: 438 Epsilon 1.0000
Episode: 23
SOLVED! Episode 23 Steps: 2 Epsilon 1.0000
Episode: 24
SOLVED! Episode 24 Steps: 3 Epsilon 1.0000
Episode: 25


SOLVED! Episode 188 Steps: 2 Epsilon 0.2988
Episode: 189
Episode: 190
SOLVED! Episode 190 Steps: 3 Epsilon 0.2988
Episode: 191
SOLVED! Episode 191 Steps: 3 Epsilon 0.2988
Episode: 192
Episode: 193
SOLVED! Episode 193 Steps: 1 Epsilon 0.2988
Episode: 194
SOLVED! Episode 194 Steps: 1 Epsilon 0.2988
Episode: 195
SOLVED! Episode 195 Steps: 7 Epsilon 0.2988
Episode: 196
SOLVED! Episode 196 Steps: 6 Epsilon 0.2988
Episode: 197
SOLVED! Episode 197 Steps: 3 Epsilon 0.2988
Episode: 198
SOLVED! Episode 198 Steps: 2 Epsilon 0.2988
Episode: 199
Episode: 200
SOLVED! Episode 200 Steps: 3 Epsilon 0.2988
Episode: 201
SOLVED! Episode 201 Steps: 3 Epsilon 0.2988
Episode: 202
Episode: 203
SOLVED! Episode 203 Steps: 1 Epsilon 0.2988
Episode: 204
SOLVED! Episode 204 Steps: 1 Epsilon 0.2988
Episode: 205
SOLVED! Episode 205 Steps: 25 Epsilon 0.2988
Episode: 206
SOLVED! Episode 206 Steps: 6 Epsilon 0.2988
Episode: 207
SOLVED! Episode 207 Steps: 3 Epsilon 0.2988
Episode: 208
SOLVED! Episode 208 Steps: 2 Epsilo

SOLVED! Episode 358 Steps: 2 Epsilon 0.2988
Episode: 359
Episode: 360
SOLVED! Episode 360 Steps: 3 Epsilon 0.2988
Episode: 361
SOLVED! Episode 361 Steps: 3 Epsilon 0.2988
Episode: 362
Episode: 363
SOLVED! Episode 363 Steps: 1 Epsilon 0.2988
Episode: 364
SOLVED! Episode 364 Steps: 1 Epsilon 0.2988
Episode: 365
SOLVED! Episode 365 Steps: 7 Epsilon 0.2988
Episode: 366
SOLVED! Episode 366 Steps: 6 Epsilon 0.2988
Episode: 367
SOLVED! Episode 367 Steps: 3 Epsilon 0.2988
Episode: 368
SOLVED! Episode 368 Steps: 2 Epsilon 0.2988
Episode: 369
Episode: 370
SOLVED! Episode 370 Steps: 3 Epsilon 0.2988
Episode: 371
SOLVED! Episode 371 Steps: 3 Epsilon 0.2988
Episode: 372
Episode: 373
SOLVED! Episode 373 Steps: 1 Epsilon 0.2988
Episode: 374
SOLVED! Episode 374 Steps: 1 Epsilon 0.2988
Episode: 375
SOLVED! Episode 375 Steps: 7 Epsilon 0.2988
Episode: 376
SOLVED! Episode 376 Steps: 6 Epsilon 0.2988
Episode: 377
SOLVED! Episode 377 Steps: 13 Epsilon 0.2988
Episode: 378
SOLVED! Episode 378 Steps: 2 Epsilo

SOLVED! Episode 524 Steps: 1 Epsilon 0.2988
Episode: 525
SOLVED! Episode 525 Steps: 37 Epsilon 0.2988
Episode: 526
SOLVED! Episode 526 Steps: 6 Epsilon 0.2988
Episode: 527
SOLVED! Episode 527 Steps: 3 Epsilon 0.2988
Episode: 528
SOLVED! Episode 528 Steps: 2 Epsilon 0.2988
Episode: 529
Episode: 530
SOLVED! Episode 530 Steps: 3 Epsilon 0.2988
Episode: 531
SOLVED! Episode 531 Steps: 3 Epsilon 0.2988
Episode: 532
SOLVED! Episode 532 Steps: 93 Epsilon 0.2988
Episode: 533
SOLVED! Episode 533 Steps: 1 Epsilon 0.2988
Episode: 534
SOLVED! Episode 534 Steps: 1 Epsilon 0.2988
Episode: 535
SOLVED! Episode 535 Steps: 7 Epsilon 0.2988
Episode: 536
SOLVED! Episode 536 Steps: 6 Epsilon 0.2988
Episode: 537
SOLVED! Episode 537 Steps: 3 Epsilon 0.2988
Episode: 538
SOLVED! Episode 538 Steps: 2 Epsilon 0.2988
Episode: 539
Episode: 540
SOLVED! Episode 540 Steps: 3 Epsilon 0.2988
Episode: 541
SOLVED! Episode 541 Steps: 3 Epsilon 0.2988
Episode: 542
Episode: 543
SOLVED! Episode 543 Steps: 1 Epsilon 0.2988
Epi

KeyboardInterrupt: 